# Peak spacing of Majorana energy oscillations as a function of $E_Z$

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
from cmath import sqrt
from scipy import optimize, sparse
import logging
import cPickle as pickle
import kwant
import dolfin
import electrostatics as estat
logging.getLogger('FFC').setLevel(logging.WARNING)

In [ ]:
"""Set system dimensions and constants"""

hbar = 1.055e-34
m = 9.109e-31
meff = 0.014*m
akwant, aFEM = 2.5e-9, 5e-9
eV, e = 1.6023e-19, 1.6023e-19
eps0 = 8.854e-12
gamma=hbar**2/(2*meff*akwant**2)
energy_par = estat.EnergyParameters(akwant=akwant, eps_dielec=8.0, eps_wire=17.7, 
                                    gamma=hbar**2/(2*meff*akwant**2),
                                    scaler=eV/gamma, bandgap=0.1, Delta=0.5e-3, 
                                    poisson_scale=e*aFEM**2/(akwant**3*eps0),
                                    miliscaler = 1000*gamma/eV)

dimension_par = estat.DimensionParameters(radius=50e-9/aFEM, top=250e-9/aFEM, 
                                          top_sc=187e-9/aFEM - 10*np.sin(np.pi/3.0),
                                          side=300e-9/aFEM, dielec_thickness=30e-9/aFEM, 
                                          L=2000)
solver_kwargs = {'alpha': 0.5, 'M': 5, 'verbose': 0, 'f_tol': 1e-7, 'maxiter': 200} 


environment_parameters = [{'V_G': 0.3011, 'V_SC': 0.0, 'l_SO': 233e-9, 'tag': '0V, 233nm'},
                         {'V_G': -0.906, 'V_SC': 0.2, 'l_SO': 233e-9, 'tag': '02V, 233nm'},
                         {'V_G': 0.3042, 'V_SC': 0.0, 'l_SO': 100e-9, 'tag': '0V, 100nm'},
                         {'V_G': -0.899, 'V_SC': 0.2, 'l_SO': 100e-9, 'tag': '02V, 100nm'},
                         {'V_G': 0.3083, 'V_SC': 0.0, 'l_SO': 60e-9, 'tag': '0V, 60nm'},
                         {'V_G': -0.89, 'V_SC': 0.2, 'l_SO': 60e-9, 'tag': '02V, 60nm'}]

In [ ]:
"""Define functions to compute Majorana peak spacings"""

def array_derivative(dx_arr, dy_arr):
    """ Compute the derivative of one array with respect to the other """
    dx = dx_arr[1] - dx_arr[0]
    dy = np.diff(dy_arr)/dx
    dx_shifted = []
    y_shifted = []
    for i in range(len(dx_arr) - 1):
        dx_shifted.append(0.5*(dx_arr[i] + dx_arr[i + 1]))
        y_shifted.append(0.5*(dy_arr[i] + dy_arr[i + 1]))
    return np.array(y_shifted), np.array(dx_shifted), np.array(dy)

def partial_dk_dEz_derivative(E, Ez, akwant, l_SO):
    """ Evaluate the analytical expression for the partial derivative of k_F
        with respect to E_Z """
    numerator = 2*Ez/(sqrt(2))
    sqrt_term_inner = sqrt(4*Ez**2 + 16*(akwant/l_SO)**4 + 16*(akwant/l_SO)**2*E)
    sqrt_term_outer = sqrt(4*(akwant/l_SO)**2 + 2*E + sqrt_term_inner)
    denominator = sqrt_term_inner*sqrt_term_outer
    dk_dEz_partial = numerator/denominator
    return dk_dEz_partial.real

def partial_dk_dmu_derivative(E, Ez, akwant, l_SO):
    """ Evaluate the analytical expression for the partial derivative of k_F
        with respect to mu """
    sqrt_term_inner = sqrt(4*Ez**2 + 16*(akwant/l_SO)**4 + 16*(akwant/l_SO)**2*E)
    sqrt_term_outer = sqrt(4*(akwant/l_SO)**2 + 2*E + sqrt_term_inner)
    dk_dmu = 1/(2*sqrt(2)*sqrt_term_outer)*(16*(akwant/l_SO)**2/(2*sqrt_term_inner) + 2)
    return dk_dmu

def dk_dEz_derivative(Ez_range, mu_range, akwant, l_SO):
    """ Compute the complete derivative of k_F with respect to E_Z """
    mu_shifted, Ez_range_shifted, dmu_dEz_arr = array_derivative(Ez_range, mu_range)
    partial_dk_dEz_arr = np.array([partial_dk_dEz_derivative(a, b, akwant, l_SO)\
                                   for a, b in zip(mu_shifted, Ez_range_shifted)])
    dk_dmu_arr = [partial_dk_dmu_derivative(a, b, akwant, l_SO)\
                  for a, b in zip(mu_shifted, Ez_range_shifted)]
    dk_dEz = partial_dk_dEz_arr + np.array([a*b for a, b in zip(dk_dmu_arr, dmu_dEz_arr)])
    return dk_dEz    

def peak_spacing(dk_dEz_arr, L, akwant):
    """ Compute the peak spacing in terms of E_Z """
    delta_E = np.array([akwant*np.pi/(x*L*1e-9) for x in dk_dEz_arr])
    return delta_E

def local_maxima(x):
    return signal.argrelextrema(np.array(x), np.greater)[0]

def compute_center(x, y):
    return (x + y)/2.0

def compute_peak_distance(x, y):
    return abs(y - x)

def transform_coord_to_value(a1, a2):
    values = []
    for i in range(len(a1)):
        values.append(a2[a1[i]])
    return values

def find_peak_distance_vs_Ez(array):
    Ez_strength = []
    peak_spacing = []
    for i in range(len(array) - 1):
        center = compute_center(array[i], array[i+1])
        peak_distance = compute_peak_distance(array[i], array[i + 1])
        Ez_strength.append(center)
        peak_spacing.append(peak_distance)
    return Ez_strength, peak_spacing

class LuInv(sparse.linalg.LinearOperator):
    def __init__(self, A):

        inst = kwant.linalg.mumps.MUMPSContext()
        inst.analyze(A, ordering='metis')
        inst.factor(A)
        self.solve = inst.solve
        sparse.linalg.LinearOperator.__init__(self, A.dtype, A.shape) 

    def _matvec(self, x):
        return self.solve(x.astype(self.dtype))


def majo_splitting(mu, Ez, alpha, energy_par, dimension_par, a=2.5):
    """Compute splittings of Majorana fermions

    Parameters:
    ----------
    mu : float
        chemical potential in units of hbar^2/2 m a^2
    Ez : float
        Zeeman splitting in units of hbar^2/2 m a^2
    alpha : float
        spin-orbit splitting, alpha is 1/lso in units of a
    a : float
        lattice spacing in nanometers
    L : float
        wire length in nanometers
    energy_par: SimpleNamespace
        Namespace containing energy scales and material constants

    Returns:
    --------
    eigvals[0] : float
        Lowest eigenvalue in units of hbar^2/2ma^2
    eigvals[2] : float
        Third eigenvalue in units of hbar^2/2ma^2
    eigvals[4] : float
        Fifth eigenvalue in units of hbar^2/2ma^2
    """
    scaler = energy_par.scaler
    Delta_scaled = scaler*energy_par.Delta
    L = dimension_par.L

    sigma0 = np.array([[1, 0], [0, 1]])
    sigmax = np.array([[0, 1], [1, 0]])
    sigmay = np.array([[0, -1j], [1j, 0]])
    sigmaz = np.array([[1, 0], [0, -1]])

    lat = kwant.lattice.chain(a)
    sys = kwant.Builder()

    def wire(pos):
        x = pos[0]
        return 0.0 < x < L

    sys[lat.shape(wire, (a,))] = ((2 - mu) * np.kron(sigma0, sigmaz) + Ez *
                                  np.kron(sigmax, sigma0) + Delta_scaled *
                                  np.kron(sigma0,sigmax))
    sys[lat.neighbors()] = (-1 * np.kron(sigma0, sigmaz) +
                            1j * alpha * np.kron(sigmay, sigmaz))

    sys = sys.finalized()
    ham = sys.hamiltonian_submatrix(sparse=True)
    eigvals = abs(sparse.linalg.eigs(ham, sigma=0,
                                     k=20, OPinv=LuInv(ham))[0]).real

    return eigvals[0], eigvals[2], eigvals[4]

## Peak spacings and splitting oscillations from the array of $\mu$ vs. $E_Z$ values

In [ ]:
with open("data_mu_single_band.p", 'rb') as fp:
    Ez_range = pickle.load(fp)
    mu_pert_dict = pickle.load(fp)
    
peak_spacing_dict = {}
majo_coupling_dict = {}

kw_str = ['0V, 233nm', '02V, 233nm', '0V, 100nm', '02V, 100nm', '0V, 60nm', '02V, 60nm']
l_so_dict = {'0V, 233nm': 233e-9, '02V, 233nm': 233e-9, '0V, 100nm': 100e-9,\
             '02V, 100nm': 100e-9, '0V, 60nm': 60e-9, '02V, 60nm': 60e-9}

for i, kw in enumerate(kw_str):
    l_SO = l_so_dict[kw]
    mu_shifted, Ez_range_shifted, mu_derivative = array_derivative(Ez_range,
                                                                   np.array(mu_pert_dict[kw]))
    dk_dmu_arr = [partial_dk_dmu_derivative(a, b, akwant, l_SO) for a, b\
                  in zip(mu_shifted, Ez_range_shifted)]

    dk_dEz = dk_dEz_derivative(Ez_range, np.array(mu_pert_dict[kw]), akwant, l_SO)
    peak_spacing_arr = peak_spacing(dk_dEz, dimension_par.L, akwant)
    Ez_range_scaled = 1000*gamma/eV*Ez_range_shifted
    peak_spacing_dict[kw] = 1000*gamma/eV*peak_spacing_arr
    
    majo_coupling_list = []
    for j, Ez in enumerate(Ez_range):
        mu = mu_pert_dict[kw][j]
        majo_energy = majo_splitting(mu, Ez, akwant/l_SO, energy_par, dimension_par)[0]
        majo_coupling_list.append(majo_energy)
    majo_coupling_dict[kw] = energy_par.miliscaler*np.array(majo_coupling_list)

In [ ]:
peak_spacing_dict_cut = {}
for kw in kw_str:    
    Ez_range_cut = []
    peak_spacing_cut = []
    for i, Ez in enumerate(Ez_range_scaled):
        if Ez > 1:
            Ez_range_cut.append(Ez)
            peak_spacing_cut.append(peak_spacing_dict[kw].real[i])
    peak_spacing_dict_cut[kw] = peak_spacing_cut
    
clr_str = {'0V, 233nm': 'b-', '02V, 233nm': 'b--', '0V, 100nm': 'r-',\
           '02V, 100nm': 'r--', '0V, 60nm': 'k-', '02V, 60nm': 'k--'}
kw_str_0V = ['0V, 233nm', '0V, 100nm', '0V, 60nm']
title_str = ['$V_{SC} = 0$ V, $l_{SO} = 233$ nm', '$V_{SC} = 0.2$ V, $l_{SO} = 233$ nm',\
             '$V_{SC} = 0$ V, $l_{SO} = 100$ nm', '$V_{SC} = 0.2$ V, $l_{SO} = 100$ nm',\
             '$V_{SC} = 0$ V, $l_{SO} = 60$ nm','$V_{SC} = 0.2$ V, $l_{SO} = 60$ nm']


fig = plt.figure(figsize=(16, 8))
for i, kw in enumerate(kw_str):
    plt.plot(Ez_range_cut, peak_spacing_dict_cut[kw], clr_str[kw], lw=4, 
             label=title_str[i])

plt.legend(loc='upper left', fontsize=15)
plt.xlim(0,7)
plt.ylim(0, 2.3)
plt.ylabel('$\Delta E_{Z, peak}$ [meV]', fontsize=24)
plt.xlabel('$E_Z$ [meV]', fontsize=24)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)

ax_inset = fig.add_axes([0.55, 0.55, 0.35, 0.35])
for i, kw in enumerate(kw_str_0V):
    ax_inset.plot(1000*gamma/eV*Ez_range, majo_coupling_dict[kw], clr_str[kw], lw=2)
ax_inset.set_ylim(0, 0.13)
ax_inset.set_xlim(0, 7)
ax_inset.locator_params(axis='y', nbins=3)
ax_inset.text(0.2, 0.8, '$V_{SC} = 0$ V', transform=ax_inset.transAxes, fontsize=26)
ax_inset.tick_params(labelsize=15)
ax_inset.set_ylabel('$\Delta E$ [meV]', fontsize=15)
ax_inset.set_xlabel('$E_Z$ [meV]', fontsize=15)
plt.show()

In [ ]:
with open("data_peak_spacing.p", 'wb') as fp:
    pickle.dump(peak_spacing_dict, fp)
    pickle.dump(majo_coupling_dict, fp)
    pickle.dump(Ez_range, fp)
    pickle.dump(Ez_range_scaled, fp)